## 13 · Resume training from checkpoint  
Ray automatically resumes from the latest checkpoint if one is available. This makes fault tolerance seamless—if the system interrupts a node or it fails, training can pick up where it left off with no manual intervention.

In [ ]:
# 13. Demonstrate fault-tolerant resume
result = trainer.fit()
print("Metrics after resume run:", result.metrics)

### 14 · Inference helper — remote GPU forecast  
Define a Ray remote function that loads a model checkpoint on a spare GPU and generates a forecast given a recent input window. Ray distributes this function to any available resource and runs independently of the training workers, making it great for batch inference or serving.

In [ ]:
# 14. Inference helper – forecast next 12 h from best checkpoint

@ray.remote(num_gpus=1)
def forecast_from_checkpoint(ckpt_path, past_array):
    ck = Checkpoint.from_directory(ckpt_path)
    model = TimeSeriesTransformer(INPUT_WINDOW, HORIZON, d_model=128, nhead=4, num_layers=3)
    with ck.as_directory() as d:
        sd = torch.load(os.path.join(d, "model.pt"), map_location="cuda")
        sd = {k.replace("module.", "", 1): v for k,v in sd.items()}
        model.load_state_dict(sd)
    model.eval().cuda()

    past = torch.tensor(past_array, dtype=torch.float32).unsqueeze(0).unsqueeze(-1).cuda()
    with torch.no_grad():
        pred_norm = model(past).cpu().squeeze().numpy()
    return pred_norm